<a href="https://colab.research.google.com/github/nmcardoso/splus-website/blob/master/sdss_stamps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Image Options:
# p = ["G", "L", "P", "S", "O", "B", "F", "M", "Q", "I", "A", "X"]
# f = ["Grid", "Label", "PhotoObjs", "SpecObjs", "Outline",
#     "BoundingBox", "Fields", "Masks", "Plates", "InvertImage", "APOGEE", "2MASS Images"];

In [0]:
# url_format: http://skyserver.sdss.org/dr15/SkyServerWS/ImgCutout/getjpeg?TaskName=Skyserver.Chart.Navi&ra=229.525575753922&dec=42.7458537608544&scale=0.3&width=300&height=300&opt=X

In [0]:
import requests
from IPython.display import Image, display
import pandas as pd
import shutil
import os
import tarfile
from google.colab import drive
import copy

In [0]:
drive.mount('/gdrive')

Mounted at /gdrive


In [0]:
base_url = 'http://skyserver.sdss.org/dr15/SkyServerWS/ImgCutout/getjpeg'
downloads_path = '/content/downloads'
dataset_path = '/gdrive/My Drive/ml_datasets/splus_crossmatch.csv'
data = None

In [0]:
def load_dataset(force=False):
  global data
  if (not data or force):
    data = pd.read_csv(dataset_path)

def get_params(ra, dec, scale=0.2, width=200, height=200, opt=''):
  return {
      'TaskName': 'Skyserver.Chart.Navi',
      'ra': ra,
      'dec': dec,
      'scale': scale,
      'width': width,
      'height': height,
      'opt': opt
  }

def prepare_downloads_dir(classes):
  shutil.rmtree(downloads_path)
  os.mkdir(downloads_path)
  for c in classes:
    os.mkdir(f'{downloads_path}/{c}')

def download_image(url, params, filename, output_path=downloads_path):
  resp = requests.get(url, params)
  if (resp.status_code == 200):
    with open(f'{output_path}/{filename}', 'wb') as f:
      f.write(resp.content)
  else:
    raise Exception(f'Error: {resp.status_code} [{filename}]')

def make_tarfile(source, output):
  with tarfile.open(output, "w:gz") as tar:
    tar.add(source, arcname=os.path.basename(source))
  print(f'Tarfile created successfully [{output}]')

def batch_download(data, classes_column, limit=None):
  classes = list(final_data.groupby(classes_column).indices.keys())
  prepare_downloads_dir(classes)
  total = limit if limit else data.shape[0]
  i = 0
  for index, row in data.iterrows():
    if (limit and i >= limit):
      break

    try:
      download_image(
          base_url, 
          get_params(row['RA_2'], row['Dec_2'], scale=0.15), 
          f'{row["ID"]}.jpg',
          output_path=f'{downloads_path}/{row[classes_column]}'
      )
      print(f'Success [{row["ID"]}] ({(((i + 1)/total)*100):.2f}%)')
    except Exception as error:
      print(error)

    i += 1

In [0]:
load_dataset()

print(*data.columns, sep='\n')

In [0]:
kmap = {}
for i, r in data.iterrows():
  klass = r['gz2class']
  kmap[klass] = kmap[klass] + 1 if klass in kmap else 1

sorted_kmap = sorted(kmap.items(), key=lambda x: x[1], reverse=True)
print(*[f'{x[0]}: {x[1]}' for x in sorted_kmap[:50]], sep='\n')

Ei: 6166
Er: 4294
Ec: 1683
Ser: 388
Sc2m: 316
Sc?t: 245
Ei(m): 157
SBc2m: 139
Er(m): 130
Sen: 126
Sc2t: 123
Sc2l: 121
Ei(i): 103
Sc?m: 91
Er(o): 90
SBc2l: 71
Sb2m: 69
Ei(o): 68
Sb: 66
Sb(r): 56
Sc: 52
SBb2m: 50
Sc3m: 44
Sd(i): 40
Sb2l: 40
Sb?t: 35
SBb: 32
Sb2t: 31
SBb2t: 30
Sc3t: 29
SBc: 29
SBc2t: 26
Sc1t: 22
Sc(i): 22
Sc1m: 21
Sd2m: 20
Sd2l: 18
Er(i): 17
Ei(r): 16
A: 13
SBb2l: 12
Sd?t: 12
Sd: 12
Er(r): 12
SBd2m: 12
SBb?t: 11
Sc1l: 11
Ec(m): 11
Sb?m: 11
Sc+t: 10


In [0]:
valid_classes = ['Ei', 'Er', 'Ec', 'Ser', 'Sc2m']

final_data = data[data['gz2class'].isin(valid_classes)]
final_data.describe()

,specobjid,dr8objid,dr7objid,ra_1,dec_1,total_classifications,total_votes,t01_smooth_or_features_a01_smooth_count,t01_smooth_or_features_a01_smooth_weight,t01_smooth_or_features_a01_smooth_fraction,t01_smooth_or_features_a01_smooth_weighted_fraction,t01_smooth_or_features_a01_smooth_debiased,t01_smooth_or_features_a01_smooth_flag,t01_smooth_or_features_a02_features_or_disk_count,t01_smooth_or_features_a02_features_or_disk_weight,t01_smooth_or_features_a02_features_or_disk_fraction,t01_smooth_or_features_a02_features_or_disk_weighted_fraction,t01_smooth_or_features_a02_features_or_disk_debiased,t01_smooth_or_features_a02_features_or_disk_flag,t01_smooth_or_features_a03_star_or_artifact_count,t01_smooth_or_features_a03_star_or_artifact_weight,t01_smooth_or_features_a03_star_or_artifact_fraction,t01_smooth_or_features_a03_star_or_artifact_weighted_fraction,t01_smooth_or_features_a03_star_or_artifact_debiased,t01_smooth_or_features_a03_star_or_artifact_flag,t02_edgeon_a04_yes_count,t02_edgeon_a04_yes_weight,t02_edgeon_a04_yes_fraction,t02_edgeon_a04_yes_weighted_fraction,t02_edgeon_a04_yes_debiased,t02_edgeon_a04_yes_flag,t02_edgeon_a05_no_count,t02_edgeon_a05_no_weight,t02_edgeon_a05_no_fraction,t02_edgeon_a05_no_weighted_fraction,t02_edgeon_a05_no_debiased,t02_edgeon_a05_no_flag,t03_bar_a06_bar_count,t03_bar_a06_bar_weight,t03_bar_a06_bar_fraction,...,t11_arms_number_a32_2_debiased,t11_arms_number_a32_2_flag,t11_arms_number_a33_3_count,t11_arms_number_a33_3_weight,t11_arms_number_a33_3_fraction,t11_arms_number_a33_3_weighted_fraction,t11_arms_number_a33_3_debiased,t11_arms_number_a33_3_flag,t11_arms_number_a34_4_count,t11_arms_number_a34_4_weight,t11_arms_number_a34_4_fraction,t11_arms_number_a34_4_weighted_fraction,t11_arms_number_a34_4_debiased,t11_arms_number_a34_4_flag,t11_arms_number_a36_more_than_4_count,t11_arms_number_a36_more_than_4_weight,t11_arms_number_a36_more_than_4_fraction,t11_arms_number_a36_more_than_4_weighted_fraction,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a36_more_than_4_flag,t11_arms_number_a37_cant_tell_count,t11_arms_number_a37_cant_tell_weight,t11_arms_number_a37_cant_tell_fraction,t11_arms_number_a37_cant_tell_weighted_fraction,t11_arms_number_a37_cant_tell_debiased,t11_arms_number_a37_cant_tell_flag,RA_2,Dec_2,X,Y,z_petro,i_petro,r_petro,g_petro,FWHM,MUMAX,A,B,PROB_GAL,Separation
count,1.284700e+04,1.256000e+04,1.284700e+04,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.0,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,...,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.0,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.0,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000,12847.000000
mean,5.631999e+17,1.237658e+18,5.878301e+17,149.446839,-0.010881,44.918113,159.104149,34.667082,34.491701,0.769213,0.779385,0.766252,0.546275,8.043512,7.959876,0.181884,0.183328,0.199890,0.018604,2.207519,1.653197,0.048886,0.037282,0.036352,0.0,2.416907,2.362078,0.196941,0.190042,0.190042,0.033237,5.628007,5.598889,0.784222,0.788475,0.788475,0.041255,0.892037,0.880144,0.115315,...,0.154720,0.015957,0.107963,0.106604,0.014136,0.013008,0.013008,0.000078,0.034950,0.033840,0.007226,0.006584,0.006584,0.0,0.034405,0.033533,0.013184,0.012598,0.012598,0.0,0.902156,0.898174,0.297659,0.295435,0.295435,0.

In [0]:
batch_download(final_data, 'gz2class')

Success [SPLUS.STRIPE82-0099.13710.griz] (0.01%)
Success [SPLUS.STRIPE82-0099.18853.griz] (0.02%)
Success [SPLUS.STRIPE82-0101.10970.griz] (0.02%)
Success [SPLUS.STRIPE82-0101.13310.griz] (0.03%)
Success [SPLUS.STRIPE82-0101.17108.griz] (0.04%)
Success [SPLUS.STRIPE82-0101.18989.griz] (0.05%)
Success [SPLUS.STRIPE82-0101.19089.griz] (0.05%)
Success [SPLUS.STRIPE82-0103.17908.griz] (0.06%)
Success [SPLUS.STRIPE82-0103.11839.griz] (0.07%)
Success [SPLUS.STRIPE82-0103.18950.griz] (0.08%)
Success [SPLUS.STRIPE82-0103.09685.griz] (0.09%)
Success [SPLUS.STRIPE82-0103.18223.griz] (0.09%)
Success [SPLUS.STRIPE82-0103.13744.griz] (0.10%)
Success [SPLUS.STRIPE82-0103.15392.griz] (0.11%)
Success [SPLUS.STRIPE82-0103.15231.griz] (0.12%)
Success [SPLUS.STRIPE82-0103.12462.griz] (0.12%)
Success [SPLUS.STRIPE82-0103.15574.griz] (0.13%)
Success [SPLUS.STRIPE82-0103.10375.griz] (0.14%)
Success [SPLUS.STRIPE82-0103.15655.griz] (0.15%)
Success [SPLUS.STRIPE82-0105.11679.griz] (0.16%)
Success [SPLUS.STRIP

In [0]:
make_tarfile(downloads_path, '/gdrive/My Drive/ml_datasets/sdss_stripe82.tar.gz')

Tarfile created successfully [/gdrive/My Drive/ml_datasets/sdss_stripe82.tar.gz]
